In [50]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [51]:
# import the libraries :
from keras.layers import *
from keras.models import *
from numpy import asarray
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# handling the warning :
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn

In [52]:
# read the dataset and find the train, test set size : ws = 2

def read_data(file_path):
    data = pd.read_csv(file_path)
    # drop all the infinite values :
    data = data.dropna()
    data = data.values
    #data = get_the_modified_dataset(file_path,ws,data)
    n = data.shape[1] - 1
    x = data[:,0:n]
    y = data[:,n]

    return x,y

file_path = './gdrive/My Drive/MSC Thesis ROOT FOLDER/MSC_THESIS_NEW_START/prev_features_train_9.csv'
X_train,y_train = read_data(file_path)
file_path = './gdrive/My Drive/MSC Thesis ROOT FOLDER/MSC_THESIS_NEW_START/prev_features_test_9.csv'
X_test,y_test = read_data(file_path)

# perform normalization :
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(27001, 75) (27001,)
(14090, 75) (14090,)


In [53]:
print('without undersampling : ')
print('Train residue statistics : ')
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )

print('Test residue statistics : ')
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )


without undersampling : 
Train residue statistics : 
positive class :  1013
negative class :  25988
positive ratio : negative ratio  3.8979529013390795  :  100.0
Test residue statistics : 
positive class :  507
negative class :  13583
positive ratio : negative ratio  3.7326069351395126  :  100.0


In [54]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42,replacement=True)
rus_X_train,rus_y_train = rus.fit_resample(X_train,y_train)

print('with undersampling : ')
print('Train residue statistics : ')
unique_elements, counts_elements = np.unique(rus_y_train, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )

print('Test residue statistics : ')
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )



with undersampling : 
Train residue statistics : 
positive class :  1013
negative class :  1013
positive ratio : negative ratio  100.0  :  100.0
Test residue statistics : 
positive class :  507
negative class :  13583
positive ratio : negative ratio  3.7326069351395126  :  100.0


In [55]:
# import libraries for CNN :

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [56]:
# make the residue into image :
ws = 4
def residue_to_image(ws,X):
    
    start_index = ws ;
    end_index =  len(X) - 1  - ws ;
    
    print('start index : ',start_index)
    print('end index : ',end_index)

    X_images = np.zeros((1,X.shape[1]),dtype=float)
     
    
    for i in range(start_index,end_index+1):
        temp_X = X[i-ws:i+ws+1,:]
          
     
        X_images = np.concatenate( (X_images,temp_X),axis=0 )
        
    
    return X_images          

print('Train information : ')
X_images = residue_to_image(ws,rus_X_train)
X_train_images = np.delete(X_images,1,axis=0)
print('X_train image : ',X_train_images.shape) 

print('Test information : ')
X_images = residue_to_image(ws,X_test)
X_test_images = np.delete(X_images,1,axis=0)
print('X_test image : ',X_test_images.shape)  


Train information : 
start index :  4
end index :  2021
X_train image :  (18162, 75)
Test information : 
start index :  4
end index :  14085
X_test image :  (126738, 75)


In [57]:
y_train_images = rus_y_train[ws : ( len(rus_X_train) - 1 - ws ) + 1]
print('y_train image ',y_train_images.shape) 

y_test_images = y_test[ws : ( len(y_test) - 1 - ws ) + 1]
print('y_test image ',y_test_images.shape) 


y_train image  (2018,)
y_test image  (14082,)


In [58]:
X_train_images = X_train_images.reshape(rus_X_train.shape[0] - ws*2,2*ws+1,rus_X_train.shape[1])
X_test_images = X_test_images.reshape(X_test.shape[0] - ws*2,2*ws+1,X_test.shape[1])

print('My Dataset dataset : ')
print('My Dataset - X_train : ',X_train_images.shape)
print('My Dataset - y_train : ',y_train_images.shape)
print('My Dataset - X_test : ',X_test_images.shape)
print('My Dataset - y_test : ',y_test_images.shape)


My Dataset dataset : 
My Dataset - X_train :  (2018, 9, 75)
My Dataset - y_train :  (2018,)
My Dataset - X_test :  (14082, 9, 75)
My Dataset - y_test :  (14082,)


In [59]:
# Reshape the image so that it can work with the keras API :
X_train_images = X_train_images.reshape(X_train_images.shape[0],2*ws+1,X_train_images.shape[2],1)
X_test_images = X_test_images.reshape(X_test_images.shape[0],2*ws+1,X_test_images.shape[2],1)

print('X_train_images : ',X_train_images.shape)
print('X_test_images : ',X_test_images.shape)

X_train_images :  (2018, 9, 75, 1)
X_test_images :  (14082, 9, 75, 1)


In [60]:
import numpy
from tensorflow import keras
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import initializers

In [69]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

def create_model(kernel_size_1=0,no_of_kernel=0,kernel_size_2 =0,pool_kernel_size=0,size_each_dense_layer_1=0,dropout_rate_1=0.0,size_each_dense_layer_2=0,dropout_rate_2=0.0,learning_rate=0.0):
	  # create model
    model = Sequential()
    
    model.add( Conv2D(filters=no_of_kernel,kernel_size=kernel_size_1,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu',
            input_shape=(2*ws+1,X_train_images.shape[2],1) ) )
    model.add( Conv2D(filters=no_of_kernel,kernel_size=kernel_size_2,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu') ) 
    model.add( MaxPool2D( pool_size=pool_kernel_size) )
    model.add(Flatten())
    model.add(Dense(units=size_each_dense_layer_1,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=dropout_rate_1))
    model.add(Dense(units=size_each_dense_layer_2,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=dropout_rate_2))
    model.add(Dense(1, activation='sigmoid'))
    
    #model.summary()
    plot_model(model, to_file='model_plot.png') 
    
    model.compile( optimizer=keras.optimizers.Adam(learning_rate),loss='binary_crossentropy', metrics='accuracy' )
    return model
    

In [70]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)


In [72]:
 # define the grid search parameters

kernel_size_1 = list(range(2,4));
no_of_kernel = list(range(12,100,15));
kernel_size_2 = list(range(2,4));

pool_kernel_size = list(range(2,4));

size_each_dense_layer_1 = list(range(12,100,15));
dropout_rate_1 = [0.3,0.5,0.7,0.9];

size_each_dense_layer_2 = list(range(12,100,15));
dropout_rate_2 = [0.3,0.5,0.7,0.9];

learning_rate = [1e-2, 1e-3];


param_grid = dict(kernel_size_1=kernel_size_1,no_of_kernel=no_of_kernel,
                  kernel_size_2=kernel_size_2,

                  pool_kernel_size=pool_kernel_size,
                  
                  size_each_dense_layer_1=size_each_dense_layer_1,
                  dropout_rate_1=dropout_rate_1,
                  
                  size_each_dense_layer_2=size_each_dense_layer_2,
                  dropout_rate_2=dropout_rate_2,
                  
                  learning_rate=learning_rate)

import time

start_time = time.time()
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=10,scoring='f1')
grid_result = grid.fit(X_train_images, y_train_images)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_) )
end_time = time.time()

print('Time required for random search : ',(end_time-start_time)/60 , ' minutes')

       

Best: 0.483948 using {'size_each_dense_layer_2': 12, 'size_each_dense_layer_1': 42, 'pool_kernel_size': 2, 'no_of_kernel': 57, 'learning_rate': 0.01, 'kernel_size_2': 3, 'kernel_size_1': 2, 'dropout_rate_2': 0.3, 'dropout_rate_1': 0.3}
Time required for random search :  7.905468479792277  minutes


In [73]:
'''
consider accuracy : 
'no_of_kernel': 87,
'kernel_size_1': 3,
'no_of_kernel': 87,
'kernel_size_2': 2,
'pool_kernel_size': 3,
'size_each_dense_layer_1': 72,
'dropout_rate_1': 0.3,
'size_each_dense_layer_2': 57, 
'dropout_rate_2': 0.3,
'learning_rate': 0.001,
'''

# consider f1 score: 
no_of_kernel = 57
kernel_size_1 = 2
kernel_size_2 = 3
pool_kernel_size = 2
size_each_dense_layer_1 = 42
dropout_rate_1 = 0.3
size_each_dense_layer_2 = 12 
dropout_rate_2 = 0.3
learning_rate = 0.01



In [74]:
model = Sequential()
model.add( Conv2D(filters=no_of_kernel,kernel_size=kernel_size_1,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu',
            input_shape=(2*ws+1,X_train_images.shape[2],1) ) )
model.add( Conv2D(filters=no_of_kernel,kernel_size=kernel_size_2,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu') ) 
model.add( MaxPool2D( pool_size=pool_kernel_size) )
model.add(Flatten())
model.add(Dense(units=size_each_dense_layer_1,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
model.add(Dropout(rate=dropout_rate_1))
model.add(Dense(units=size_each_dense_layer_2,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
model.add(Dropout(rate=dropout_rate_2))
model.add(Dense(1, activation='sigmoid'))
    
#model.summary()
plot_model(model, to_file='model_plot.png') 
model.compile(optimizer=keras.optimizers.Adam(learning_rate),loss='binary_crossentropy', metrics='accuracy')



In [75]:
model.fit( X_train_images, y_train_images, epochs=10, validation_data=(X_test_images,y_test_images) )

Epoch 1/10
64/64 [==============================] - 18s 267ms/step - loss: 0.5938 - accuracy: 0.6302 - val_loss: 0.9783 - val_accuracy: 0.6008
Epoch 2/10
64/64 [==============================] - 17s 264ms/step - loss: 0.2422 - accuracy: 0.9080 - val_loss: 2.3017 - val_accuracy: 0.3815
Epoch 3/10
64/64 [==============================] - 17s 263ms/step - loss: 0.1778 - accuracy: 0.9302 - val_loss: 3.9047 - val_accuracy: 0.3740
Epoch 4/10
64/64 [==============================] - 17s 268ms/step - loss: 0.1275 - accuracy: 0.9385 - val_loss: 1.9112 - val_accuracy: 0.5614
Epoch 5/10
64/64 [==============================] - 18s 280ms/step - loss: 0.1484 - accuracy: 0.9287 - val_loss: 3.6846 - val_accuracy: 0.5075
Epoch 6/10
64/64 [==============================] - 17s 270ms/step - loss: 0.1277 - accuracy: 0.9443 - val_loss: 3.9158 - val_accuracy: 0.3716
Epoch 7/10
64/64 [==============================] - 17s 271ms/step - loss: 0.1130 - accuracy: 0.9571 - val_loss: 3.8529 - val_accuracy: 0.3514

In [76]:
from sklearn.metrics import confusion_matrix
y_pred = np.round(    model.predict(X_test_images), 0  )
print( confusion_matrix(y_test_images, y_pred ) )

[[4436 9139]
 [ 103  404]]


In [77]:
tn, fp, fn, tp = confusion_matrix(y_test_images, y_pred).ravel()
print(tn, fp, fn, tp)

# accuracy :
acc = (tp + tn) / (tp + fp + fn + tn)
print('Accuracy : ',"{:.2f}".format(acc) )

# sensitivity :
sen = tp / (tp + fn)
print('Sensitivity : ',"{:.2f}".format(sen) )

# specificity :
spec = tn / (fp + tn)
print('Specificity : ',"{:.2f}".format(spec) )


# mcc :
import math
mcc = (tp * tn - fp * fn) / math.sqrt( (tp+fp) * (fn+tn) * (fp+tn) * (tp+fn) )
print('MCC : ',"{:.2f}".format(mcc) ) 

# f1-score :
# recall -> sen
pre = tp/(tp+fp)
f1_score = (2*pre*sen) / (pre + sen)
print('f1-score : ',"{:.2f}".format(f1_score) )


4436 9139 103 404
Accuracy :  0.34
Sensitivity :  0.80
Specificity :  0.33
MCC :  0.05
f1-score :  0.08
